In [1]:
from pyspark.sql import SparkSession

from pyspark.ml.feature import VectorAssembler

from pyspark.ml.classification import LogisticRegression

In [ ]:
#pyspark init
builder = SparkSession.Builder().appName('creditcard_highAPI')
spark = builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/03/28 22:07:23 WARN Utils: Your hostname, DESKTOP-0H87CFM resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/28 22:07:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/28 22:07:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Read input file
raw_df = spark.read.csv('creditcard.csv', header=True, inferSchema=True)

In [4]:
#Extract features name
cols = [col for col in raw_df.columns if col != 'Class']

#Use VectorAssembler to transform feature columns into a single vector
assembler = VectorAssembler(inputCols=cols, outputCol= 'features')
data = assembler.transform(raw_df).select('features', 'Class')

In [ ]:
#Split data into training and testing sets

###training set proportion parameter:
train_size = 0.7
###

train, test = data.randomSplit([train_size, 1 - train_size], seed=24) #Fixed with seed for reproductivity

In [36]:
classifier = LogisticRegression(featuresCol= 'features', labelCol= 'Class')
model = classifier.fit(train)

In [37]:
summary = model.summary
print('Accuracy: ', summary.accuracy)
print('AUC: ', summary.areaUnderROC)
print('Precision: ', summary.precisionByLabel)
print('Recall: ', summary.recallByLabel)

Accuracy:  0.9991931051419083


AUC:  0.9794501302671836
Precision:  [0.9993209146577059, 0.8942307692307693]
Recall:  [0.9998711370399006, 0.6158940397350994]


In [38]:
res = model.transform(test)

In [39]:
res.groupby('Class', 'prediction').count().sort('Class', 'prediction').show()

+-----+----------+------+
|Class|prediction| count|
+-----+----------+------+
|    0|       0.0|113573|
|    0|       1.0|    18|
|    1|       0.0|    65|
|    1|       1.0|   125|
+-----+----------+------+

